In [ ]:

import pandas as pd
from datetime import datetime, timedelta
import warnings
%load_ext autoreload
%autoreload 2
from scripts.extract_data import get_wpdx_kenya , collapse_zinc_csv

df_wpdx = get_wpdx_kenya()
df_wpdx['report_date'] = pd.to_datetime(df_wpdx['report_date'])
df_wpdx['env_start_date'] = df_wpdx['report_date'] - timedelta(days=30)
df_wpdx['env_end_date'] = df_wpdx['report_date']

sample_size = 1000 
kenya_sample = df_wpdx.sample(n=sample_size, random_state=42).reset_index(drop=True)

print(f"Processing {len(kenya_sample)} water points with temporal accuracy")

✅ || wpdx_kenya.csv already exists with 21953 rows, skipping fetch.
Processing 1000 water points with temporal accuracy
